텍스트 데이터 다루기 

- 텍스트 데이터는 주로 글자가 연결된 문자열로 표현된다.
- 텍스트 데이터의 길이는 서로 같은 경우가 거의 없다. 
- 이 특성은 정량적인 연속형 특성과, 범주형 특성과는 다르므로 다른 전처리 방법을 이용해야 한다. 

    1) 문자열 데이터 타입
    
        - 범주형 데이터 : 고정된 목록으로 구성
        - 범주에 의미를 연결시킬 수 있는 임의의 문자열 : 텍스트 필드로 받는 응답중 범주형데이터에 벗어났지만 범주화시킬 수 있는 답
        
          -> 응답을 포용할 수 있는 범주를 정의하는 것이 최선의 방법 (자동화 어렵고, 수작업이 많아진다. )
          
        - 구조화된 문자열 데이터 : 주소나 장소, 사람이름, 날짜, 전화번호 등의 일정한 구조를 가진 데이터 (분석어렵고, 문맥이나 분야에 따라 다르다)
        - 텍스트 데이터 : 단어로 구성된 문장에 정보를 담고 있는 데이터 
        
    2) 예제 : 영화리뷰감성 분석
    
        - IMDb에서 수집한 영화리뷰 데이터셋 이용 ( 리뷰 리스트, 양성 / 음성 레이블 포함- 7점이사 양성, 4점이하 음성)

In [1]:
import wget

In [3]:
url ='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
wget.download(url)

100% [........................................................................] 84125825 / 84125825

'aclImdb_v1.tar.gz'

In [6]:
from sklearn.datasets import load_files

reviews_train = load_files('aclImdb_v1.tar/aclImdb/train/')

#텍스트, 레이블 포함한 Brunch 오브젝트 반환

text_train, y_train = reviews_train.data, reviews_train.target

print('text_train 타입 : ', type(text_train))
print('text_train 길이 :', len(text_train))
print('text_train[6]:\n', text_train[6])

text_train 타입 :  <class 'list'>
text_train 길이 : 75000
text_train[6]:
 b'Gloomy Sunday - Ein Lied von Liebe und Tod directed by Rolf Sch\xc3\xbcbel in 1999 is a romantic, absorbing, beautiful, and heartbreaking movie. It started like Jules and Jim; it ended as one of Agatha Christie\'s books, and in between it said something about love, friendship, devotion, jealousy, war, Holocaust, dignity, and betrayal, and it did better than The Black Book which is much more popular. It is not perfect, and it made me, a cynic, wonder in the end on the complexity of the relationships and sensational revelations, and who is who to whom but the movie simply overwhelmed me. Perfect or not, it is unforgettable. All four actors as the parts of the tragic not even a triangle but a rectangle were terrific. I do believe that three men could fell deeply for one girl as beautiful and dignified as Ilona in a star-making performance by young Hungarian actress Erica Marozs\xc3\xa1n and who would not? The titular 

In [7]:
#인덱스 6인 리뷰 출력시 HTML줄바꿈 태그가 포함되어 있어 태그 삭제

text_train = [doc.replace(b"<br />",b"") for doc in text_train]

In [8]:
#양성, 음성 레이블 수 확인
import numpy as np

print('클래스별 샘플 수 (훈련) :', np.bincount(y_train))

클래스별 샘플 수 (훈련) : [12500 12500 50000]


In [9]:
#test파일 읽기
reviews_test = load_files('aclImdb_v1.tar/aclImdb/test/')
text_test, y_test = reviews_test.data, reviews_test.target

print('테스트 데이터 문서 수 : ', len(reviews_test))
print('클래스별 샘플수 (테스트) :', np.bincount(y_test))
text_test = [doc.replace(b"<br />", b"")for doc in text_test]

테스트 데이터 문서 수 :  5
클래스별 샘플수 (테스트) : [12500 12500]


- 이 데이터 셋을 이용하여 각 리뷰의 텍스트 내용을 보고 양성인지 음성인지 구분하는 이진분류 수행할 것
- 텍스트 데이터는 머신러닝 모델이 다룰 수 있는 형태가 아니므로 텍스트 문자열 표현을 수치 표현으로 변환해야한다.

3) 텍스트 데이터를 BOW로 표현하기
    - BOW(Bag of Word)는 가장 간단하고 널리 쓰이는 방법이다.
    - 장, 문단, 문장, 서식 같은 입력 텍스트의 구조 대부분을 잃고, 각 단어가 이 말뭉치에 있는 텍스트에
    
      얼마나 많이 나타나는지만 헤아린다. ( 단어의 출현 빈도 카운트 )
    - 원본 문자열의 단어의 순서는 BOW 특성표현에서 완전히 무시된다. 
    - BOW의 세단계  
        1) 토큰화 : 각 문서에 포함된 단어(토큰)로 나눈다. (공백이나, 구두점을 기준으로 분리 )
        2) 어휘사전 구축 : 모든 문서에 나타난 모든 단어의 어휘를 모으고 번호를 매긴다.
        3) 인코딩 : 어휘 사전의 단어가 문서마다 몇번이나 나타나는지 헤아린다. 

In [11]:
#샘플 데이터에 BOW 적용하기
#BOW표현은 CountVectorizer변환기 인터페이스로 구현되어 있다.

bards_words = ["The fool doth think he is wise, ",
              "but the wise man knows himself to be a fool"]

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
vect.fit(bards_words)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

- CountVectorizer의 fit 메서드는 훈련 데이터를 토큰으로 나누고 어휘 사전을 구축하여 vocabulary_ 속성에 저장한다.


In [12]:
print('어휘 사전 크기 :', len(vect.vocabulary_))
print('어휘 사전 내용 :\n', vect.vocabulary_)

어휘 사전 크기 : 13
어휘 사전 내용 :
 {'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}


In [13]:
#훈련 데이터에 대해 BOW 표현 만들려면 transform 메서드 호출
bag_of_words = vect.transform(bards_words)
print('BOW :', repr(bag_of_words))

BOW : <2x13 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>


- BOW표현은 0이 아닌 값만 저장하는 Scipy 희소행렬로 저장되어 있다. 
- 각각의 행은 하나의 데이터 포인트를 나타내고, 각 특성은 어휘 사전에 있는 각 단어에 대응한다. 
- 대부분의 문서는 어휘 사전에 있는 단어 중 일부만 포함하므로, 특성 배열의 대부분의 원소가 0이라 희소 행렬을 사용한다.
- 값이 0인 원소를 모두 저장하는 것은 메모리 낭비라서 기피해야 한다. 
- 희소 행렬의 실제 내용을 보려면 toarray 메서드를 사용하여 밀집된 Numpy 배열로 바꿔야 한다.

In [14]:
print('BOW의 밀집 표현 : \n' , bag_of_words.toarray())

BOW의 밀집 표현 : 
 [[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


- 각 단어의 출현 횟수는 0 아니면 1이다. 즉 bards_words에 있는 두 문자열 모두 같은 단어를 두 개 이상 가지고 있지 않다. 

In [15]:
#영화 리뷰에 대한 BOW

vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
print('X_train : \n', repr(X_train))

X_train : 
 <75000x127229 sparse matrix of type '<class 'numpy.int64'>'
	with 10315468 stored elements in Compressed Sparse Row format>


-  X_train의 크기는 75000*127229로, 이 어휘 사전은 단어를 10315468개 담고 있다.
- Scipy희소 행렬로 저장되어 있고,  get_feature_name 메서드를 통해 특성에 해당하는 단어를 리스트로 반환한다.

In [17]:
feature_names = vect.get_feature_names()
print('특성 갯수 :', len(feature_names))
print('처음 20개 특성 :\n', feature_names[:20])
print('20010 ~20030까지 특성 :\n', feature_names[20010:20030])
print('매 2000번째 특성 :\n', feature_names[::2000])

특성 갯수 : 127229
처음 20개 특성 :
 ['00', '000', '0000', '0000000000000000000000000000000001', '0000000000001', '000000001', '000000003', '00000001', '000001745', '00001', '0001', '00015', '0002', '0007', '00083', '000ft', '000s', '000th', '001', '002']
20010 ~20030까지 특성 :
 ['cetait', 'cetera', 'cetin', 'cetmes', 'cetnik', 'cetniks', 'ceusescu', 'ceuta', 'ceylon', 'ceylonese', 'ceylons', 'cezanne', 'cezary', 'cezmi', 'cf', 'cfdc', 'cfm', 'cfto', 'cg', 'cga']
매 2000번째 특성 :
 ['00', '9out', 'ages', 'andress', 'aryeman', 'baio', 'bellah', 'bloodstain', 'briers', 'calm', 'cessation', 'circenses', 'complementing', 'countedsix', 'dahan', 'dench', 'discourages', 'dreamkeeeper', 'elderly', 'esposito', 'fang', 'flaquer', 'frizzyhead', 'gerry', 'grandness', 'halycon', 'hesitated', 'hubiriffic', 'incongruence', 'ireperable', 'journal', 'kindsa', 'landingham', 'limply', 'maaaybbbeee', 'marthe', 'mephestophelion', 'modem', 'mushing', 'nigger', 'oghris', 'oxbridge', 'pensamentos', 'pleaaaaaaaase', 'prettifi

- 앞의 결과에는 의미가 없을 것 같은 숫자들이 단어로 추출되었다. ( 리뷰 어딘가에 포함되어 있기 때문 )
- 의미없는 단어 중에서 의미 있는 것을 선별해내는 일은 쉽지않다.
- 또한 단어의 단수, 복수형이 서로 다른 단어로 어휘 사전에 포함되어 있다. ( 이런 것은 개별적이 아닌 함께 기록이 유리 )
- 특성 추출 방법 개선 전, 분류기를 만들어 성능 수치 확인

In [20]:
#BOW 분류기 - LogisticRegression 과 같은 선형 모델의 성능이 가장 뛰어나다.
#교차 검증이용한 LogisticRegression 모델 성능 평가

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

scores = cross_val_score(LogisticRegression(), X_train, y_train, cv = 5)
print('교차 검증 평균 점수 : ', np.mean(scores))

c:\users\cndak\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\cndak\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\cndak\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


교차 검증 평균 점수 :  0.71016


In [ ]:
#그리드 서치를 이용하여 매개변수 C의 최적값 탐색
from sklearn.model_selection import GridSearchCV

param_grid = {'C':[0.001,0.01,0.1,1,10]}

grid = GridSearchCV(LogisticRegression(), param_grid, cv = 5)
grid.fit(X_train ,y_train)

In [22]:
print('최상 교차 검증 점수 :', grid.best_score_)
print('최적 매개변수 :', grid.best_params_)

최상 교차 검증 점수 : 0.71672
최적 매개변수 : {'C': 0.1}


In [23]:
#이 매개변수로 테스트 셋의 일반화 성능 측정
X_test = vect.transform(text_test)
print('테스트 점수 :', grid.score(X_test, y_test))

테스트 점수 : 0.1314


- 일반화 성능이 매우 떨어져 단어 추출 방법을 개선해야 한다. 
- CountVectorizer는 정규 표현식을 사용해 토큰을 추출한다. 
  
  (기본적인 정규 표현 식 '\b\w\w+\b')
- 경계(\b)가 구분되고 적어도 둘 이상의 문자나 숫자(\w)가 연속된 단어를 찾는다고 이해하면 된다. 
- doesn't 같은 축약형이나 bit.ly같은 단어는 분리되고 h8ter는 한단어로 매칭된다. 
- CounterVectorizer는 모든 단어를 소문자로 바꾼다. 
- 앞선 숫자로만 표현된 의미없는 특성을 줄이는 방법은 적어도 두 개의 문서(또는 다섯 개의 문서 등)에 나타난
  
  토큰만을 이용하는 것이다. (하나의 문서만에 나타난 토큰은 테스트 세트에 나타날 가능성이 적어 도움이 안된다 )
  

In [26]:
#min_df 매개변수로 토큰이 나타날 최소 문서 개수 지정 가능
vect = CountVectorizer(min_df = 10).fit(text_train)
X_train  = vect.transform(text_train)
print('min_df로 제한한 X_train :', repr(X_train))

min_df로 제한한 X_train : <75000x31913 sparse matrix of type '<class 'numpy.int64'>'
	with 10104459 stored elements in Compressed Sparse Row format>


- 토큰이 약 2만개 가량 줄어들었다. 

In [27]:
feature_names = vect.get_feature_names()

print('처음 50개 특성 :\n', feature_names[:50])
print('20010 ~20030까지 특성 :\n', feature_names[20010:20030])
print('매 2000번째 특성 :\n', feature_names[::2000])

처음 50개 특성 :
 ['00', '000', '007', '00s', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '1000', '1001', '100th', '101', '102', '103', '104', '105', '106', '107', '108', '109', '10p', '10quality', '10rated', '10s', '10th', '10x', '11', '110', '110th', '111', '112', '1138', '115', '117', '11th', '12', '120', '1200', '125', '12th', '13', '130', '135']
20010 ~20030까지 특성 :
 ['org', 'organ', 'organic', 'organically', 'organisation', 'organisations', 'organise', 'organised', 'organism', 'organisms', 'organization', 'organizations', 'organize', 'organized', 'organizer', 'organizers', 'organizes', 'organizing', 'organs', 'orgasm']
매 2000번째 특성 :
 ['00', 'attaching', 'bummed', 'conformist', 'dimensional', 'exacerbated', 'gibberish', 'hyping', 'kovacs', 'mercies', 'order', 'printed', 'rivaled', 'slavic', 'syncing', 'unpopular']


- 확실히 숫자가 줄어들고 희귀한 단어와 철자가 틀린 단어들이 사라졌다. 

In [28]:
#그리드 서치를 이용해 모델 성능확인
grid.fit(X_train, y_train)
print('최적 교차 검증 점수 :', grid.best_score_)

c:\users\cndak\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\cndak\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\cndak\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


최적 교차 검증 점수 : 0.7114933333333333


- 교차검증 점수가 이전과 별로 달라지지 않았다.
- 모델 성능은 높아지지 않았지만 특성의 갯수가 줄어서 속도가 빨라지고 모델 이해가 쉬워졌다.

4) 불용어

    - 의미 없는 단어를 제거하는 방법은 너무 빈번하여 유용하지 않은 단어를 제거 하는 것이다. 
    - 언어별 불용어 목록 사용 또는 너무 빈번한 단어를 제외하는 것이다. 

In [30]:
#사이킷런에서 제공하는 불용어 이용

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

print('불용어 갯수 :', len(ENGLISH_STOP_WORDS))
print('매 10번 째 불용어 : \n', list(ENGLISH_STOP_WORDS)[::10])

불용어 갯수 : 318
매 10번 째 불용어 : 
 ['mostly', 'two', 'former', 'move', 'at', 'one', 'rather', 'this', 'whole', 'five', 'already', 'become', 'however', 'hereby', 'thru', 'part', 'yours', 'get', 'fire', 'without', 'noone', 'so', 'whereafter', 'against', 'these', 'a', 'has', 'eg', 'detail', 'per', 'upon', 'some']


In [34]:
#불용어 이용

vect = CountVectorizer(min_df=10, stop_words = 'english').fit(text_train)
X_train = vect.transform(text_train)
print('불용어 제거 X_train :\n', repr(X_train))

불용어 제거 X_train :
 <75000x31606 sparse matrix of type '<class 'numpy.int64'>'
	with 6491400 stored elements in Compressed Sparse Row format>


- 불용어 제거로 인해 단어가 약 3만 5천개 가량 줄었다.

In [35]:
#그리드 서치 재적용
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print('최상 교차 검증 점수 :', grid.best_score_)

c:\users\cndak\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\cndak\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\cndak\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


최상 교차 검증 점수 : 0.7129333333333333


- 불용어 제거 후에도 성능이나 모델 해석이 나아진 것 같지는 않다. 
- 고정된 불용어 목록은 모델이 데이터셋만 보고 불용어를 골라내기 어려운 작은 데이터셋에서나 도움이된다. 
- 다른 방식으로 CountVectorizer의 max_df옵션을 지정하여 자주 나타나는 단어를 제거하고 특성 개수와 성능에 영향을 주는지 연습해야 할 것으로 판단된다. 